# Effect of repealing Child Tax Credit with Child Dividend

This identifies beneficiaries of the Child Tax Credit by modeling its repeal. Both repeal from current (2017) state and TCJA state are considered on a static basis. Change to after-tax income by decile and share of after-tax income held by top 10% are calculated.

*Data: CPS  |  Tax year: 2018  |  Type: Static  |  Author: Max Ghenis  |  Date run: 2018-02-20*

## Setup

### Imports

In [2]:
import taxcalc as tc
import pandas as pd
import numpy as np
import copy
from bokeh.io import show, output_notebook
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import urllib as url_lib  # On Python 3.6 use "import urllib.request as url_lib".

In [3]:
tc.__version__

'0.16.1'

In [4]:
sns.set_style('white')
DPI = 300
mpl.rc('savefig', dpi=DPI)
mpl.rcParams['figure.dpi']= DPI

### Create policies

Load reforms from GitHub. Policies and reforms are named according to this convention:

**`{reform}_{year}_{object}`**

For example:

* `noctc_2017_policy`: Policy without CTC using 2017 law.
* `ubi_2018_calc`: Calculator replacing CTC with UBI using 2018 (current) law.

In [7]:
# Folders where reforms live.
GITHUB_BASE_URL = 'https://raw.githubusercontent.com/'

TAXCALC_GITHUB_BASE_URL = (GITHUB_BASE_URL +
                           'open-source-economics/Tax-Calculator/master/' +
                           'taxcalc/reforms/')

def read_url(url):
    return url_lib.urlopen(url).read()

def read_reform_taxcalc_github(reform_name):
    return read_url(TAXCALC_GITHUB_BASE_URL + reform_name + '.json')

def policy_from_reform(reform):
    pol = tc.Policy()
    pol.implement_reform(reform['policy'])
    if pol.reform_errors:
        print(pol.reform_errors)
    return pol

def create_static_policy_taxcalc_github(reform_name):
    reform = tc.Calculator.read_json_param_objects(
        read_reform_taxcalc_github(reform_name), None)
    return policy_from_reform(reform)

In [8]:
y2017_policy = create_static_policy_taxcalc_github(
    '2017_law')

In [10]:
noctc_reform = {
    2018: {
        '_CTC_c': [0],
        '_DependentCredit_Child_c': [0]
    }}
noctc_2017_policy = copy.deepcopy(y2017_policy)
noctc_pol_2017.implement_reform(noctc_reform)
noctc_pol_2018 = tc.Policy()  # Can't combine with next step.
noctc_pol_2018.implement_reform(noctc_reform)

### Specify `Calculator` objects for static analyses

In [11]:
recs = tc.Records.cps_constructor()

In [12]:
def static_baseline_calc(year):
    calc = tc.Calculator(records=recs, policy=tc.Policy())
    calc.advance_to_year(year)
    calc.calc_all()
    return calc

In [18]:
def weighted_sum(df, col):
    return (df[col] * df['s006']).sum()

In [55]:
def child_ubi_reform(amount):
    return {2018: {'_UBI_u18': [amount],
                   '_UBI_ecrt': [1.0]}}

In [58]:
def static_calc(use_2017_law=False, 
                ctc_treatment='keep',
                child_ubi_amount=0,
                year=2018,
                cols=['s006', 'aftertax_income', 'expanded_income', 'nu18']):
    """Creates static Calculator.

    Args:
        use_2017_law: Whether to use 2017 law vs. current law. Defaults to False.
        ctc_treatment: How the Child Tax Credit is treated. Options include:
            * 'keep': No change. Default.
            * 'repeal': End entirely.
            * 'rev_neutral_ubi': Replace with revenue-neutral child UBI.
            * 'no_cut_ubi': [NOT YET IMPLEMENTED] 
                Replace with a child UBI equal to the current maximum value.
        year: Year to advance calculations to.
        cols: Columns to extract per Calculator record. 
            Defaults to ['s006', 'expanded_income', 'aftertax_income', 'nu18'].
        
    Returns:
        DataFrame with `cols` and percentile, decile, and quintile of after-tax income.
    """
    # Initiate policy using either 2017 or current law.
    if use_2017_law:
        pol = copy.deepcopy(y2017_policy)
    else:
        pol = tc.Policy()
    # Enact reform based on ctc_treatment.
    # Repeal CTC unless it's kept.
    if ctc_treatment != 'keep':
        pol.implement_reform(noctc_reform)
    if child_ubi_amount > 0:
        pol.implement_reform(child_ubi_reform(child_ubi_amount))
    # Calculate. This is needed to calculate the revenue-neutral UBI.
    calc = tc.Calculator(records=recs, policy=pol)
    calc.advance_to_year(year)
    calc.calc_all()
    # TODO: Calculate revenue for revenue-neutral UBI.
    # Create DataFrame and add identifiers.
    df = calc.dataframe(cols)
    df['use_2017_law'] = use_2017_law
    df['ctc_treatment'] = ctc_treatment
    return df

In [67]:
scenarios_pre_ubi = pd.concat([
    static_calc(use_2017_law=False, ctc_treatment='keep'),
    static_calc(use_2017_law=False, ctc_treatment='repeal'),
    static_calc(use_2017_law=True, ctc_treatment='keep'),
    static_calc(use_2017_law=True, ctc_treatment='repeal')])

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


In [68]:
scenarios_pre_ubi.head()

,s006,aftertax_income,expanded_income,nu18,use_2017_law,ctc_treatment
0,239.46,42287.547869,52280.201254,0.0,False,keep
1,202.59,20461.070670,18176.438090,2.0,False,keep
2,255.35,96222.033747,96222.033747,0.0,False,keep
3,178.36,26597.379609,26002.668632,1.0,False,keep
4,270.80,41631.335096,41631.335096,0.0,False,keep


Calculate child allowance.

In [69]:
aftertax_income_summary = (
    scenarios_pre_ubi.groupby(['use_2017_law', 'ctc_treatment'], as_index=False).
    apply(lambda x: 
          pd.Series({
              'aftertax_income': weighted_sum(x, 'aftertax_income'), 
              'nu18': weighted_sum(x, 'nu18')}))).reset_index()
nu18_total = aftertax_income_summary['nu18'][0]

In [62]:
aftertax_income_chg = aftertax_income_summary.pivot(
    index='use_2017_law',
    columns='ctc_treatment',
    values='aftertax_income'
)
aftertax_income_chg['chg'] = aftertax_income_chg['keep'] - aftertax_income_chg['repeal']
aftertax_income_chg['child_ubi'] = aftertax_income_chg['chg'] / nu18_total
aftertax_income_chg

ctc_treatment,keep,repeal,chg,child_ubi
use_2017_law,,,,
False,1.095007e+13,1.082995e+13,1.201240e+11,1464.524234
True,1.081362e+13,1.076178e+13,5.184284e+10,632.056117


In [79]:
scenarios = pd.concat([
    scenarios_pre_ubi,
    static_calc(use_2017_law=False, ctc_treatment='rev_neutral_ubi',
                child_ubi_amount=aftertax_income_chg.loc[False, 'child_ubi']),
    static_calc(use_2017_law=True, ctc_treatment='rev_neutral_ubi',
                child_ubi_amount=aftertax_income_chg.loc[True, 'child_ubi'])
])

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


,s006,aftertax_income,expanded_income,nu18,use_2017_law,ctc_treatment
0,239.46,42287.547869,52280.201254,0.0,False,keep
1,202.59,20461.070670,18176.438090,2.0,False,keep
2,255.35,96222.033747,96222.033747,0.0,False,keep
3,178.36,26597.379609,26002.668632,1.0,False,keep
4,270.80,41631.335096,41631.335096,0.0,False,keep


## Analysis

* Add percentile to previous steps.
* Should there be a field for more "gross" income? Without benefits and UBI.

In [87]:
scenarios.loc[1000, :].sort_values('use_2017_law')

,s006,aftertax_income,expanded_income,nu18,use_2017_law,ctc_treatment
1000,14.39,80743.440366,75425.491384,3.0,False,keep
1000,14.39,78513.210786,75425.491384,3.0,False,repeal
1000,14.39,82906.783490,79819.064088,3.0,False,rev_neutral_ubi
1000,14.39,80527.250786,75425.491384,3.0,True,keep
1000,14.39,78527.250786,75425.491384,3.0,True,repeal
1000,14.39,80423.419136,77321.659734,3.0,True,rev_neutral_ubi
